# Agents: Content Generators and Reasoning Engines
## Introduction
In the fascinating world of artificial intelligence, LangChain and LLMs have opened up new horizons in data analysis, information synthesis, and content generation. Central to their functionality is the concept of Agents - intelligent systems that utilize LLMs to determine actions and facilitate complex tasks. In this way, LLMs are used more as a reasoning engine or a planner and less as content generators per-se. We discuss two primary ways we can harness the capabilities of LMMs: as content generators and as reasoning engines.

LLMs leverage their internal knowledge as content generators to create engaging and creative content from scratch. On the other hand, used as reasoning engines, they act as proficient synthesizers of information, extracting and summarizing relevant data from a multitude of sources and planning the next actions to take. Both these approaches have distinct advantages and challenges, with the choice largely dependent on the specific requirements of the task.

### Agents
In the context of language models, agents are used to decide the course of action and the sequence of these actions. These actions can either be the utilization of a tool, observing its output, or offering a response to the user. The real potential of agents unfolds when they are utilized appropriately. This explanation aims to simplify the usage of agents via the highest-level API.

Before diving into the practical usage, it's crucial to understand the following terms:

- Tool: A function that performs a specific task. It can be a Google Search, a Database lookup, a Python REPL, or other chains. A tool's interface is typically a function that takes a string as an input and returns a string as an output.
- Language Learning Model: The language model that powers the agent.
- Agent: The agent to use, identified by a string that references a supported agent class. It’s what orchestrates the LLM and the tools. This explanation focuses on using the standard supported agents via the highest-level API. For custom agent implementation, refer to the appropriate documentation.

Agents in LangChain play a crucial role in the decision-making and execution of tasks based on user input. They evaluate the situation and decide on the appropriate tools to use, if necessary.

Presently, most of the agents in LangChain fall into one of these two categories:

- "Action Agents": These agents determine and execute a single action. They are typically used for straightforward tasks.
- "Plan-and-Execute Agents": These agents first devise a plan comprising multiple actions and then execute each action sequentially. They are more suited for complex or long-running tasks as they help maintain focus on long-term objectives.

While Action Agents are more traditional and suitable for smaller tasks, Plan-and-Execute Agents help maintain long-term objectives and focus. However, they might lead to more calls and higher latency. Often, it's beneficial to let an Action Agent manage the execution for the Plan-and-Execute agent, thus utilizing both strengths.

For example, a high-level workflow of Action Agents would look something like this:

- The agent receives user input.
- It decides which tool to use (if any) and determines its input.
- The chosen tool is called with the provided input, and an observation (the output of the tool) is recorded.
- The history of the tool, tool input, and observation are relayed back to the agent, which then decides the next step.
- This process is repeated until the agent no longer needs to use a tool, at which point it directly responds to the user.

The most critical abstraction to understand is the agent itself. In the context of LangChain, the term "agents" pertains to the concept of employing a language model as a reasoning mechanism and linking it with the key element - a tool.

Tools are instrumental in connecting the language model with other sources of data or computation, including search engines, APIs, and other data repositories. Language models can only access the knowledge they've been trained on, which can quickly become obsolete. Therefore, tools are essential as they allow the agent to retrieve and incorporate current data into the prompt as context. Tools can also execute actions (like running code or modifying files) and observe the results, subsequently informing the language model's decision-making process.

## Answering Questions using an LLM as a reasoning engine

In [1]:
import os

#os.environ["OPENAI_API_KEY"] = "<YOUR-OPENAI-API-KEY>"
#os.environ["GOOGLE_API_KEY"] = "<YOUR-GOOGLE-SEARCH-API-KEY>"
#os.environ["GOOGLE_CSE_ID"] = "<YOUR-CUSTOM-SEARCH-ENGINE-ID>"

# Importing necessary modules
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI

# Loading the language model to control the agent
llm = OpenAI(model="text-davinci-003", temperature=0)

# Loading some tools to use. The llm-math tool uses an LLM, so we pass that in.
tools = load_tools(["google-search", "llm-math"], llm=llm)

# Initializing an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# Testing the agent
query = "What's the result of 1000 plus the number of goals scored in the soccer world cup in 2018?"
response = agent.run(query)
print(response)

Could not import azure.core python package.




> Entering new AgentExecutor chain...
 I need to find out the number of goals scored in the 2018 soccer world cup
Action: Google Search
Action Input: "number of goals scored in 2018 soccer world cup"
Observation: Sep 22, 2023 ... This was three more goals than the previous tournament in 2018. Goals galore. The highest ever goals to game ratio was set during the 1954 World ... Jan 31, 2020 ... A total of 169 goals were scored at the group and knockout stages of the FIFA World Cup held in Russia from the 14th of June to the 15th of July ... Aug 28, 2023 ... This number can be attributed to some unusually high-scoring games, which included Austria 7 Switzerland 5 and Hungary 9 South Korea 0. The ... Number of goals scored in the matches played between the teams in question;; Fair play points in all group matches (only one deduction could be applied to a ... France. Winner. 7. Matches played. 14. Goals. 1.7. Goal Average. 12. Yellow Cards. 0. Red Cards. TOURNAMENT AWARDS. Previous. Best 

There were 169 goals scored in the soccer world cup in 2018, so the final answer is correct.

In the example, the agent leverages its "reasoning engine" capabilities to generate responses. Instead of creating new content (acting as a content generator), the agent uses the tools at its disposal to gather, process, and synthesize information. The entire output was truncated, and the agent skillfully employed the LLM-math tool.

### Agent as a new Content Creator

The agent would create new content as a content generator rather than just pulling and processing existing information. Let's imagine a scenario where we want the agent to write a short science fiction story based on a given prompt.

We could initialize the agent with a language model and set its temperature parameter to a higher value to encourage more creativity in its outputs. It is not required to use external tools, as the agent generates content rather than retrieving or processing it.

The language model will generate a long science fiction story about interstellar explorers based on the patterns it learned during training.

In [2]:
# Importing necessary modules
from langchain.agents import initialize_agent, AgentType
from langchain.llms import OpenAI
from langchain.agents import Tool
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

prompt = PromptTemplate(
    input_variables=["query"],
    template="You're a renowned science fiction writer. {query}"
)

# Initialize the language model
llm = OpenAI(model="text-davinci-003", temperature=0)
llm_chain = LLMChain(llm=llm, prompt=prompt)

tools = [
    Tool(
        name='Science Fiction Writer',
        func=llm_chain.run,
        description='Use this tool for generating science fiction stories. Input should be a command about generating specific types of stories.'
    )
]

# Initializing an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# Testing the agent with the new prompt
response = agent.run("Compose an epic science fiction saga about interstellar explorers")
print(response)



> Entering new AgentExecutor chain...
 I should use a tool to help me generate a story
Action: Science Fiction Writer
Action Input: Generate a story about interstellar explorers
Observation: .

The year was 2120 and the human race had finally achieved interstellar travel. After decades of research and development, a team of brave explorers had been assembled to embark on a mission to explore the unknown reaches of the universe.

The team was led by Captain John Smith, a veteran of many space missions. He was joined by a diverse group of scientists, engineers, and adventurers, all eager to explore the unknown.

The team's first destination was a distant star system known as Alpha Centauri. After months of travel, they arrived at the system and began to explore the planets and moons in the vicinity.

The team discovered a number of strange and fascinating phenomena, from strange alien life forms to mysterious energy fields. They also encountered a number of hostile forces, from hostile

Here, the Agent is primarily using its internal knowledge to generate the output. Here's a brief explanation of how that works:

The agent receives a prompt to "Compose an epic science fiction saga about interstellar explorers.”
The agent then uses its understanding of language, narrative structure, and the specific themes mentioned in the prompt (science fiction, interstellar exploration, etc.) to generate a story.
LLM's understanding comes from its training data. It was trained on a diverse range of internet text, so it has a broad base of information to draw from. When asked to generate a science fiction story, it uses patterns it learned during training about how such stories are typically structured and what elements they usually contain.

Remember, even though the language model has vast training data to draw from, it doesn't "know" specific facts or have access to real-time information. Its responses are generated based on patterns learned during training, not from a specific knowledge database.

## A Code Example of Using BabyAGI

Although BabyAGI uses specific vector stores and model providers, one of the benefits of implementing it with LangChain is that you can easily swap those out for different options. In this implementation, we use a FAISS vector store.

In [3]:
import os
#os.environ["OPENAI_API_KEY"] = "<YOUR-OPENAI-API-KEY>"
from langchain.embeddings import OpenAIEmbeddings
import faiss
from langchain.vectorstores import FAISS
from langchain.docstore import InMemoryDocstore

# Define the embedding model
embeddings_model = OpenAIEmbeddings(model="text-embedding-ada-002")

# Initialize the vectorstore
embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})

Execute a call to BabyAGI:

In [4]:
from langchain import OpenAI
from langchain.experimental import BabyAGI

# set the goal
goal = "Plan a trip to the Grand Canyon"

# create thebabyagi agent
# If max_iterations is None, the agent may go on forever if stuck in loops
baby_agi = BabyAGI.from_llm(
    llm=OpenAI(model="text-davinci-003", temperature=0),
    vectorstore=vectorstore,
    verbose=False,
    max_iterations=3
)
response = baby_agi({"objective": goal})


*****TASK LIST*****

1: Make a todo list

*****NEXT TASK*****

1: Make a todo list

*****TASK RESULT*****



1. Research the best time to visit the Grand Canyon
2. Book flights to the Grand Canyon
3. Book a hotel near the Grand Canyon
4. Research the best activities to do at the Grand Canyon
5. Research the best restaurants near the Grand Canyon
6. Pack necessary items for the trip
7. Make a budget for the trip
8. Make a list of places to visit at the Grand Canyon
9. Make a list of souvenirs to buy
10. Make a list of photos to take

*****TASK LIST*****

2: Research the best places to view the Grand Canyon
3: Research the best places to take photos at the Grand Canyon
4: Research the best places to take a tour at the Grand Canyon
5: Research the best places to take a helicopter tour at the Grand Canyon
6: Research the best places to hike at the Grand Canyon
7: Research the best places to camp at the Grand Canyon
8: Research the best places to buy souvenirs at the Grand Canyon
9: Resear

from langchain import OpenAI
from langchain.experimental import BabyAGI

# set the goal
goal = "Plan a trip to the Grand Canyon"

# create thebabyagi agent
# If max_iterations is None, the agent may go on forever if stuck in loops
baby_agi = BabyAGI.from_llm(
    llm=OpenAI(model="text-davinci-003", temperature=0),
    vectorstore=vectorstore,
    verbose=False,
    max_iterations=3
)
response = baby_agi({"objective": goal})